# Create Annotations

This program creates an xml file and copy of an image for training machine learning models.
This program opens a directory and for each image in the folder:
- displays it
- Allows the user to drag the mouse to create rectangles of the areas in the image that contain items to train the model on.
- press w to write the xml file and image to go to the next image
- press + to go to the next image if there is nothing to train in the present image
-   "   1 to skip ahead 10 images
- press - to go to the previous image
-   "   e to end the program


In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install opencv-python

In [ ]:
def write_annot(obj_type, obj_annots_dir, obj_f_num, org_f_path,org_f_width,org_f_height,org_f_depth, refPts):
    # With credit to: 
    # https://www.geeksforgeeks.org/reading-writing-text-files-python/
    annot_file_path = obj_annots_dir+obj_f_num+'.xml'
    annot_file = open(annot_file_path,"w") 

    annot_file.write("<annotation>\n") 
    annot_file.write("	<folder>"+obj_type+"</folder>\n") 
    annot_file.write("	<filename>"+obj_f_num+".jpg</filename>\n") 
    annot_file.write("	<path>"+org_f_path+"</path>\n") 
    annot_file.write("	<source>\n") 
    annot_file.write("		<database>pompeiiinpictures.com</database>\n") 
    annot_file.write("	</source>\n") 
    annot_file.write("	<size>\n") 
    annot_file.write("		<width>"+str(org_f_width)+"</width>\n") 
    annot_file.write("		<height>"+str(org_f_height)+"</height>\n") 
    annot_file.write("		<depth>"+str(org_f_depth)+"</depth>\n") 
    annot_file.write("	</size>\n") 
    annot_file.write("	<segmented>0</segmented>\n") 
    for ocn in range(0,len(refPts)):
        refPt = refPts[ocn]
        refPtMin = refPt[0]
        refPtMax = refPt[1]
        annot_file.write("	<object>\n") 
        annot_file.write("		<name>"+obj_type+"</name>\n") 
        #annot_file.write("		<pose>Unspecified</pose>\n") 
        annot_file.write("		<truncated>0</truncated>\n") 
        annot_file.write("		<difficult>0</difficult>\n") 
        annot_file.write("		<bndbox>\n") 
        annot_file.write("			<xmin>"+str(refPtMin[0])+"</xmin>\n") 
        annot_file.write("			<ymin>"+str(refPtMin[1])+"</ymin>\n") 
        annot_file.write("			<xmax>"+str(refPtMax[0])+"</xmax>\n") 
        annot_file.write("			<ymax>"+str(refPtMax[1])+"</ymax>\n") 
        annot_file.write("		</bndbox>\n") 
        annot_file.write("	</object>\n") 
    annot_file.write("</annotation>\n") 
    annot_file.close() 

In [ ]:
# import the necessary packages

import cv2

# initialize the list of reference points

refPt = []
refPts = []

def click_and_crop(event, x, y, flags, param):
	# grab references to the global variables
	global refPt, cropping

	# if the left mouse button was clicked, record the starting
	# (x, y) coordinates and indicate that cropping is being
	# performed
	if event == cv2.EVENT_LBUTTONDOWN:
		refPt = [(x, y)]
		cropping = True

	# check to see if the left mouse button was released
	elif event == cv2.EVENT_LBUTTONUP:
		# record the ending (x, y) coordinates and indicate that
		# the cropping operation is finished
		refPt.append((x, y))
		cropping = False

		# draw a rectangle around the region of interest
		cv2.rectangle(resized, refPt[0], refPt[1], (0, 0, 255), 2)
		cv2.imshow("image", resized)
		refPts.append(refPt)

# Skip 

In [ ]:
# With credit to:
# Adrian Rosebrock 
# https://www.pyimagesearch.com/2015/03/09/capturing-mouse-click-events-with-python-and-opencv/

# load the image, clone it, and setup the mouse callback function

refPts = []
original_file_path = '/home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/pompeiiinpictures Ferebee 10405 May 2017 DSCN1949.JPG'
image = cv2.imread(original_file_path)
print('Original Dimensions : ',image.shape)

# With credit to for image resizing
# https://www.tutorialkart.com/opencv/python/opencv-python-resize-image/
scale_percent = 1024/image.shape[1] # percent of original size
print(scale_percent)
width = int(image.shape[1] * scale_percent )
height = int(image.shape[0] * scale_percent )
dim = (width, height)
# resize image
resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
resized_original = resized.copy()
 
print('Resized Dimensions : ',resized.shape)
 
#clone = resized.copy()
cv2.namedWindow("image")
cv2.setMouseCallback("image", click_and_crop)

# keep looping until the 'c' key is pressed
while True:
	# display the image and wait for a keypress
	cv2.imshow("image", resized)
	key = cv2.waitKey(1) & 0xFF

	# if the 'c' key is pressed, break from the loop
	if key == ord("c"):
		break

print(refPts)

# close all open windows
cv2.destroyAllWindows()

object_type = 'opus_incertum'
object_annots_dir = '/home/student/data_5000_project/data/images/construction_types/processed/test/annots/'
object_images_dir = '/home/student/data_5000_project/data/images/construction_types/processed/test/images/'
object_file_num = '00001'
write_annot(object_type, object_annots_dir, object_file_num, original_file_path,resized.shape[1],resized.shape[0],resized.shape[2], refPts)
cv2.imwrite(object_images_dir+object_file_num+".jpg",resized_original)

In [ ]:
def annotate_image(original_image_path, object_type, object_annots_dir, object_images_dir, object_file_num_int):
    global image,resized,resized_original,refPts
    # With credit to:
    # Adrian Rosebrock 
    # https://www.pyimagesearch.com/2015/03/09/capturing-mouse-click-events-with-python-and-opencv/
    object_file_num_int_return = object_file_num_int
    object_file_num = "0000"+str(object_file_num_int)
    object_file_num = object_file_num[-5:]
    print("original_image_path", original_image_path)
    # load the image, clone it, and setup the mouse callback function
    refPts = []
    image = []
    image = cv2.imread(original_image_path)
    #print('Original Dimensions 2: ',image.shape)
    #plt.imshow(imgage, cmap = 'gray', interpolation = 'bicubic')
    #plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    #plt.show()

    # With credit to for image resizing
    # https://www.tutorialkart.com/opencv/python/opencv-python-resize-image/
    scale_percent = 1024/image.shape[1] # percent of original size
    #print(scale_percent)
    width = int(image.shape[1] * scale_percent )
    height = int(image.shape[0] * scale_percent )
    dim = (width, height)
    # resize image
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    resized_original = resized.copy()
     
    #print('Resized Dimensions : ',resized.shape)
 
    #clone = resized.copy()
    #cv2.startWindowThread()
    #cv2.namedWindow("image")
    #cv2.setMouseCallback("image", click_and_crop)

    # keep looping until the 'c' key is pressed
    while True:
        # display the image and wait for a keypress
        cv2.imshow("image", resized)
        cv2.moveWindow("image", 0,100)
        key = cv2.waitKey(1) & 0xFF

        # if the 'c' key is pressed, break from the loop
        if key == ord("c"):
            object_file_num_int_return = object_file_num_int_return +1
            break
        if key == ord("w"):
            write_annot(object_type, object_annots_dir, object_file_num, original_image_path,resized.shape[1],resized.shape[0],resized.shape[2], refPts)
            cv2.imwrite(object_images_dir+object_file_num+".jpg",resized_original)
            object_file_num_int_return = object_file_num_int_return +1
            break
        if key == ord("-"):
            object_file_num_int_return = object_file_num_int_return-1
            break
        if key == ord("+"):
            object_file_num_int_return = object_file_num_int_return+1
            break
        if key == ord("1"):
            object_file_num_int_return = object_file_num_int_return+10
            break
        if key == ord("9"):
            object_file_num_int_return = object_file_num_int_return+90
            break            
        if key == ord("e"):
            object_file_num_int_return = -1
            break    

    #print(refPts)

    # close all open windows
    cv2.destroyAllWindows()
    image=None

    return object_file_num_int_return 
#object_type = 'opus_incertum'
#object_annots_dir = '/home/student/data_5000_project/data/images/construction_types/processed/annots/'
#object_images_dir = '/home/student/data_5000_project/data/images/construction_types/processed/images/'
#object_file_num = '00001'
    #write_annot(object_type, object_annots_dir, object_file_num, original_file_path,resized.shape[1],resized.shape[0],resized.shape[2], refPts)
    #cv2.imwrite(object_images_dir+object_file_num+".jpg",resized_original)

In [ ]:
# With credit to for getting just files
# https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory


raw_images_directory = '/home/student/data_5000_project/data/images/construction_types/raw/opus_incertum/'
from os import listdir
from os.path import isfile, join
raw_image_files = [f for f in listdir(raw_images_directory) if isfile(join(raw_images_directory, f))]
object_type = 'opus_incertum'
object_annots_dir = '/home/student/data_5000_project/data/images/construction_types/processed/'+object_type+'/annots/'
object_images_dir = '/home/student/data_5000_project/data/images/construction_types/processed/'+object_type+'/images/'
object_file_num_int=0
while object_file_num_int>-1:
    cv2.startWindowThread()
    #cv2_window_name = "image" + str(object_file_num_int)+" "+raw_image_files[object_file_num_int]
    cv2_window_name = "image"
    cv2.namedWindow(cv2_window_name)
    cv2.setMouseCallback(cv2_window_name, click_and_crop)
    original_image_path = raw_images_directory + raw_image_files[object_file_num_int]
    #print(original_image_path)
    #print(raw_image_files[object_file_num_int])
    object_file_num_int = annotate_image(original_image_path, object_type, object_annots_dir, object_images_dir, object_file_num_int)
    print("                                                                                                  ",object_file_num_int)
    cv2.destroyAllWindows()

In [ ]:
print(onlyfiles[0] )

# Copy boxes for use by Osama

In [ ]:
object_type = 'opus_incertum'
object_annots_dir = '/home/student/data_5000_project/data/images/construction_types/processed/'+object_type+'/annots/'
object_images_dir = '/home/student/data_5000_project/data/images/construction_types/processed/'+object_type+'/images/'
object_boxes_output_dir = '/home/student/data_5000_project/data/images/construction_types/processed/'+object_type+'/selected_training/'

import cv2
from xml.etree import ElementTree
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(object_annots_dir) if isfile(join(object_annots_dir, f))]

for fn in onlyfiles:
    #print(fn)
    tree = ElementTree.parse(object_annots_dir+fn)
    # get the root of the document
    root = tree.getroot()
    # extract each bounding box
    
    fn_image = root.find('./filename').text
    #print(fn_image)
    box_num=0
    for box in root.findall('.//bndbox'):
        box_num=box_num+1
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)
        if(ymin>ymax):
            ytemp = ymin
            ymin = ymax
            ymax=ytemp
        if(xmin>xmax):
            xtemp = xmin
            xmin = xmax
            xmax=xtemp            
        coors = [xmin, ymin, xmax, ymax]
        #print(coors)
        
        img = cv2.imread(object_images_dir+fn_image)
        crop_img = img[ymin:ymax, xmin:xmax]
        print(crop_img.shape)
        fntd = object_boxes_output_dir+"td_"+fn[:5]+"_b"+str(box_num)+fn_image[5:]
        print(fntd)
        cv2.imwrite(fntd,crop_img)
        

